In [11]:
import re
from collections import Counter, defaultdict
import numpy as np
from nltk.tokenize import word_tokenize

Вспомним, как писать бесконечные генераторы с помощью ```yield```

In [3]:
def num_generator():
    start = 0
    while True:
        yield start
        start += 1

gen = num_generator()
print(next(gen))

0
1


А если хочется получить сразу несколько первых элементов из генератора?

**Задание** Напишите функцию-генератор, которая будет выдавать элементы до тех пор, пока не наберется нужное количество (`n`).

In [5]:
def take(generator, n):
    gen = generator
    start = 0
    while start < n:
        yield next(gen)
        start += 1
        
list(take(num_generator(), 5))

[0, 1, 2, 3, 4]

**Задание** Напишите генератор, который будет выдавать только четные числа.

In [6]:
def even_num_generator(num_generator):
    gen = num_generator
    start = 0
    while True:
        if (start % 2) == 0:
            yield next(gen)
        else:
            next(gen)
        start += 1

list(take(even_num_generator(num_generator()), 5))

[0, 2, 4, 6, 8]

## N-граммная языковая модель

In [80]:
!head poetry.txt -n 5

Гудели как-то мы с друзьями ночкой тёмною,
И спьяну, сдуру я признался им в одном:
"В десанте нет мне равных, - говорю, - не стрёмно мне
Застыть на целый день в засаде под кустом!"



***Задание*** Посчитайте частотности слов в файле за ```O(1)``` памяти, то есть без ```f.read()```

In [12]:
word_counts = Counter()
with open('poetry.txt', encoding='utf-8') as f:
    for line in f:
        tokens = word_tokenize(line)
        for t in tokens:
            if re.search('[а-яА-ЯЁё]', t):
                word_counts[t.lower()] += 1

print(word_counts.most_common(100))

[('и', 111377), ('в', 75436), ('не', 50914), ('я', 38324), ('на', 34592), ('как', 26130), ('с', 25474), ('что', 21110), ('ты', 17929), ('а', 16292), ('но', 15882), ('он', 14068), ('мне', 14028), ('за', 11009), ('все', 10842), ('к', 10678), ('по', 9729), ('о', 9728), ('мы', 9312), ('так', 8698), ('от', 8687), ('у', 7679), ('из', 7599), ('меня', 7045), ('где', 6698), ('то', 5901), ('его', 5855), ('ни', 5790), ('под', 5781), ('нет', 5711), ('их', 5599), ('когда', 5592), ('же', 5409), ('над', 5303), ('это', 5257), ('мой', 5165), ('тебя', 4949), ('вот', 4824), ('только', 4692), ('она', 4672), ('да', 4665), ('без', 4643), ('до', 4551), ('бы', 4484), ('там', 4468), ('был', 4443), ('для', 4384), ('кто', 4334), ('нам', 4065), ('нас', 3964), ('ли', 3865), ('тебе', 3838), ('всё', 3832), ('вы', 3732), ('во', 3681), ('ее', 3590), ('еще', 3573), ('сердце', 3530), ('чтоб', 3436), ('лишь', 3368), ('быть', 3210), ('если', 3198), ('жизнь', 3172), ('есть', 2957), ('они', 2871), ('может', 2757), ('было', 

Для создания нашей N-граммной модели будем накапливать статистику для всех строк стихотворений. 
Генерировать стихотворения будем построчно. Для этого нужно научить генерировать "первое слово" и "последнее слово" строки.

**Задание** Напишите `parser` - функцию, которая будет выдавать поток токенов из файла. При этом каждая "строка" стихотворения должна начинаться с токена ```<BOS>``` и ```<EOS>```.
Добавьте предобработку: фильтруйте все строки длиной менее 20 символов. Удалите все знаки препинания. Приведите к нижнему регистру

In [79]:
import re
def parser(path):
    yield

Напишем N-граммную языковую модель по стишкам.

Языковая модель умеет оценивать вероятности $\mathbf{P}(w_1, \ldots, w_n) = \prod_k \mathbf{P}(w_k|w_{k-1}, \ldots, w_{1})$.

N-граммная языковая модель приближает эту вероятность, используя предположение, что вероятность токена зависит только от недавней истории: $\mathbf{P}(w_k|w_1, \ldots, w_{k-1}) = \mathbf{P}(w_k|w_{k-1}, \ldots, w_{k-N + 1})$.

Для начала нужно собрать статистику. Для простоты будем работать с биграмной моделью, а значит - нужно собрать информацию:

- о биграммных частотностях $(w_{i-1}) \to C(w_i)$
- о триграммных частотностях $(w_{i-2}, w_{i-1}) \to C(w_i)$

Также хочется сохранять в статистику n-gram информацию о начале и коцне стишка.

**Задание** Напишите функцию, которая будет из потока токенов формировать и выдавать наружу пары (ngram, next_word).

Каждый стишок должен начинаться с токена ```<BOS>``` и заканчиваться ```<EOS>``` (beginning of sequence, end of sequence).

In [46]:
def compose_ngram(tokens_stream):

    token = next(tokens_stream)
    yield
    # Подсказки:
    # Что сохраняем, если token --> <BOS>?
    # Что делаем, если token --> <EOS>?

Соберем статистику:

In [78]:
ngrams_counter = defaultdict(Counter)
for ngram in compose_ngram(parser('poetry.txt')):
    ngrams_counter[tuple(ngram[0])][ngram[1]] += 1

Теперь генерировать будем так: есть стартовый токен. Проверяем последнюю триграмму в статистике. Если есть, генерируем с помощью нее новое слово. Если нет, ищем биграму (она должна быть).
Если засемплили ```<EOS>``` --> завершаем генерацию и строку.
Генерировать будем четверостишья (стихотворения из 4 строк)

In [75]:
def sample_token(ngrams_counter, ngram):
    probs = np.array(list(ngrams_counter[ngram].values()))
    probs = probs / np.sum(probs)
    return np.random.choice(list(ngrams_counter[ngram]), p=probs)  
  
def generate_line(ngrams_counter):
    buffer = ['<BOS>']
    return buffer

In [77]:
for _ in range(4):
    print(' '.join(generate_line(ngrams_counter)).strip('<EOS> <BOS>'))

как пешеходу в зной лаская полуденный
на току молотящих снопы спозаранок
лишь в честь убитого сердца толпою
где можешь ты обжора ах злодей
